In [27]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [28]:
inpatient_specialty = pd.read_csv('/home/bhatti/dataset/VCHAMPS/inpatient_specialty_train.csv').iloc[:,1:]

In [29]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [31]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [32]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

### Merge with inpatient data
Filter out specialty time before last discharge date

In [33]:
path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,Died,AO,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,1,3,14.16,2,0,0,0,0,0,3,...,0.6,3,0,0,0,0,0,2352.34,2016-07-22 21:36:24.0,0
1,2,21,71.17,5,2,10,0,0,1,9,...,1.5,10,1,0,9,1,1,0.00,2023-09-12 02:28:10.0,0
2,3,1,1.83,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,885.57,2002-03-30 18:12:14.0,0
3,4,1,7.15,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,0.00,2015-02-23 09:46:15.0,0
4,5,1,1.04,1,0,1,0,0,1,1,...,1.0,1,0,0,0,0,0,26.75,2004-10-02 06:54:56.0,1


In [60]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,Discharge date
0,1,2016-07-22 21:36:24.0
1,2,2023-09-12 02:28:10.0
2,3,2002-03-30 18:12:14.0
3,4,2015-02-23 09:46:15.0
4,5,2004-10-02 06:54:56.0
...,...,...
84531,169055,2002-04-12 16:51:53.0
84532,169057,2025-02-12 17:23:04.0
84533,169060,2017-02-08 09:55:03.0
84534,169062,2005-06-04 03:34:07.0


In [100]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['Discharge date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,Discharge date
1,100001,86.568969,2012-07-21 23:00:00,2012-07-22 19:18:47,PM&R TRANSITIONAL REHAB,North Carolina,0 days 20:18:47,1.0,2012-07-22 19:18:48.0
2,100015,56.646451,2002-07-16 06:40:13,2002-07-16 06:40:13,INTERMEDIATE MEDICINE,Massachusetts,0 days 00:00:00,0.0,2003-10-13 00:05:54.0
4,100019,81.174959,1999-10-24 10:44:24,1999-10-31 10:20:31,PLASTIC SURGERY,Florida,6 days 23:36:07,0.0,1999-10-31 10:20:32.0
5,10002,85.020599,2011-06-19 00:41:40,2011-06-19 00:41:40,MEDICAL STEP DOWN,Ohio,0 days 00:00:00,0.0,2015-04-30 06:08:11.0
6,100026,56.667339,2012-11-08 21:53:52,2012-11-13 02:53:48,SHORT STAY GRECC-NHCU,Missouri,4 days 04:59:56,1.0,2017-04-27 03:48:54.0
...,...,...,...,...,...,...,...,...,...
1452905,99993,58.013747,2007-02-18 00:35:16,2007-02-18 04:55:16,METABOLIC,Kentucky,0 days 04:20:00,0.0,2013-08-03 22:37:59.0
1452906,99994,83.090126,2016-10-18 10:35:16,2016-10-21 11:02:57,NH SHORT STAY DEMENTIA CARE,Texas,3 days 00:27:41,1.0,2019-09-06 05:14:03.0
1452907,99994,85.951737,2019-08-30 08:38:03,2019-09-05 09:28:48,"PULMONARY, NON-TB",Texas,6 days 00:50:45,NaN,2019-09-06 05:14:03.0
1452908,99996,55.942565,2011-08-17 00:10:52,2011-08-17 00:10:52,GENERAL(ACUTE MEDICINE),Georgia,0 days 00:00:00,0.0,2013-12-31 04:17:49.0


In [41]:
inpatient_specialty3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1182463 entries, 1 to 1452909
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype          
---  ------                --------------    -----          
 0   Internalpatientid     1182463 non-null  int64          
 1   Age at specialty      1182463 non-null  float64        
 2   Specialty start date  1182463 non-null  datetime64[ns] 
 3   Specialty end date    1182463 non-null  datetime64[ns] 
 4   Specialty             1182463 non-null  object         
 5   State                 1182463 non-null  object         
 6   period                1182463 non-null  timedelta64[ns]
 7   Specialty encoded     1136851 non-null  float64        
 8   Discharge date        1182463 non-null  object         
dtypes: datetime64[ns](2), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 90.2+ MB


In [101]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_241671/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
/tmp/ipykernel_241671/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [103]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

/tmp/ipykernel_241671/1693156475.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [107]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_241671/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,1,69.437719,6
1,2,65.779856,53
2,3,78.746725,1
3,4,84.167594,1
4,5,76.412480,2
...,...,...,...
84395,169055,58.918427,4
84396,169057,82.144829,73
84397,169060,64.611030,23
84398,169062,72.476952,21


In [108]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,1,69.437719,6,1.985007,2.286435,6,0
1,2,65.779856,53,1.695425,3.832509,50,2
2,3,78.746725,1,81.500222,162.085913,0,1
3,4,84.167594,1,2.383800,1.709373,1,0
4,5,76.412480,2,1.407809,1.351702,2,0
...,...,...,...,...,...,...,...
84395,169055,58.918427,4,1.617095,1.735104,1,3
84396,169057,82.144829,73,1.277754,2.100649,57,16
84397,169060,64.611030,23,1.681511,3.024752,17,6
84398,169062,72.476952,21,6.870090,11.462599,15,6


In [109]:
inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_preprocessed.csv')